In [1]:
import numpy as np
import pandas as pd
import torch
import pickle
import re
from itertools import chain
from collections import Counter
import torch.nn as nn
import glob
import random
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.utils.class_weight import compute_class_weight
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import OneHotEncoder
import transformers
from transformers import AdamW
from transformers import DistilBertTokenizer, DistilBertModel
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler, Dataset, random_split
from tqdm import tqdm
import time

%matplotlib inline

In [2]:
# specify device
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

# Load the BERT tokenizer
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

# Bert mode
bert = DistilBertModel.from_pretrained('distilbert-base-uncased')

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_projector.weight', 'vocab_transform.bias', 'vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [4]:
class SpeciesDescriptions(Dataset):
    
    """Description dataset without species names."""
    
    def __init__(self, root_dir):
        
        self.root_dir = root_dir
        self.samples = []
        self._init_dataset()
        #self.label_encoder = LabelEncoder()
        
    def __len__(self):
        return len(self.samples)
        
    def __getitem__(self, idx):
        return self.samples[idx]
     
    def _init_dataset(self):
        
        # Encoder 
        lb = LabelBinarizer()
        # Init dict
        datadict = {}
        # Load the pickle list
        data_files = glob.glob(self.root_dir + 'subset*.pkl')
        for data_file in data_files:
            # Open the dict and update
            datadict.update(pickle.load(open(data_file, 'rb')))
            
        # Get keys and encode them
        keys = np.array([key for key in datadict.keys()])
        print(len(keys))
        keys_encoded = lb.fit_transform(keys)
        # Extract the values with the encoded keys
        self.samples += [(key_label, value) for key_label, (key, value_list) in zip(keys_encoded, datadict.items()) for value in value_list]
        
        '''
        for key_encode, (key, value_list) in zip(keys_encoded, datadict.items()):
            if len(value_list) < 10:
                continue
            for i in range(len(value_list):
                self.samples.append((key_encode, ' '.join(random.sample(value_list, 5))))
        '''

In [5]:
try:
    # Colab
    from google.colab import drive
    root = '/content/gdrive/My Drive/'
    drive.mount('/content/gdrive')
    print('Mounted @Google')
except:
    # Local
    root = "../data/processed/"
    print('Mounted @Local')

start = time.time()
# Load data
data = SpeciesDescriptions(root)
end = time.time()
print("Time consumed in working: ",end - start)

Mounted @Local
170


TypeError: sequence item 0: expected str instance, tuple found

In [5]:
total_count = len(data)
train_count = int(1.0 * total_count)
valid_count = int(0.0 * total_count)
test_count = total_count - train_count - valid_count
train_dataset, valid_dataset, test_dataset = random_split(data, (train_count, valid_count, test_count), 
                                                       generator=torch.Generator().manual_seed(33))

In [6]:
batch_size = 2

# Random sample (skewed set)
train_sampler = RandomSampler(train_dataset)
# DataLoader for train set
train_dataloader = DataLoader(train_dataset, sampler=train_sampler, batch_size=batch_size)

# Random sample
val_sampler = SequentialSampler(valid_dataset)
# DataLoader for validation set
val_dataloader = DataLoader(valid_dataset, sampler=val_sampler, batch_size=batch_size)

In [7]:
# Freeze all the parameters
for param in bert.parameters():
    param.requires_grad = True

In [8]:
class BERT(nn.Module):
    def __init__(self, bert):
        
        super(BERT, self).__init__()
        
        # Distil Bert model
        self.bert = bert
        ## Additional layers
        # Dropout layer
        self.dropout = nn.Dropout(0.3)
        # Relu 
        self.relu =  nn.ReLU()
        # Linear I 
        self.fc1 = nn.Linear(768, 512)
        # Linear II (Out)
        self.fc2 = nn.Linear(512, 170)
        # Softmax
        self.softmax = nn.LogSoftmax(dim=1)


    # Forward pass
    def forward(self, sent_id, mask):

        # Pass data trough bert and extract 
        cls_hs = self.bert(sent_id, attention_mask=mask)
        # Extract hidden state
        hidden_state = cls_hs.last_hidden_state
        # Only first is needed for classification
        pooler = hidden_state[:, 0]
        
        # Dense layer 1        
        x = self.fc1(pooler)
        # ReLU activation
        x = self.relu(x)
        # Drop out
        x = self.dropout(x)
        # Dense layer 2
        x = self.fc2(x)
        # Activation
        x = self.softmax(x)

        return x

In [86]:
# Load the entire model
model = BERT(bert)

# Load trained model (colab)
try:
    try:
        model_save_name = 'saved_weights_NLP_test.pt'
        path = F"/content/gdrive/My Drive/{model_save_name}"
        model.load_state_dict(torch.load(path))
        print('Google Success')

    except:
        model_save_name = 'saved_weights_NLP_subset.pt'
        path = "../models/" + model_save_name
        model.load_state_dict(torch.load(path, 
                                         map_location=torch.device('cpu')))
        print('Local Success')
except:
    print('No pretrained model found.')

# Push the model to GPU
model = model.to(device)

Local Success


In [10]:
# Load optimizer (Adam best for bert)
optimizer = torch.optim.Adam(params = model.parameters(), lr=3e-5)
# Define loss function
softmax = nn.Softmax(1)
CEloss = nn.CrossEntropyLoss()

def tokenize_batch(batch_set):
    
    """
    Tokenize a pytorch dataset using the hugging face tokenizer.
    """
    
    # Extract the labels and text
    y = batch_set[0]
    text = batch_set[1]
    
    
    # Tokenize the text
    tokens = tokenizer.batch_encode_plus(text,
                max_length = 512,
                padding=True,
                truncation=True)
    
    # Convert to tensors
    seq = torch.tensor(tokens['input_ids'])
    mask = torch.tensor(tokens['attention_mask'])
    
    return seq, mask, y

def train():
  
    """
    Function to train classification Bert model.
    """
    
    model.train()
    total_loss = 0
    
    # Iterate over batches
    for batch in tqdm(train_dataloader):
        
        # Tokenize batch
        train_seq, train_mask, train_y = tokenize_batch(batch)
        # Push to device
        sent_id, mask, labels = [t.to(device) for t in [train_seq, train_mask, train_y]]
        # Clear gradients 
        model.zero_grad()        
        # Get predictions
        preds = model(sent_id, mask)
        # Compute loss
        loss =  CEloss(preds, torch.max(labels, 1)[1]) 
        #loss =  CEloss(preds, labels) 
        # Update total loss
        total_loss = total_loss + loss.item()
        # Backward pass to calculate the gradients
        loss.backward()
        # Clip the the gradients to 1.0. It helps in preventing the exploding gradient problem
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        # Update parameters
        optimizer.step()

    # Compute the training loss of the epoch
    avg_loss = total_loss / len(train_dataloader)

    return avg_loss


def evaluate():
    
    """
    Function to test classification Bert model.
    """
  
    # Deactivate dropout layers
    model.eval()
    total_loss = 0

    # Iterate over batches
    for batch in tqdm(val_dataloader):   
        # Tokenize batch
        val_seq, val_mask, val_y = tokenize_batch(batch)
        # Push to device
        sent_id, mask, labels = [t.to(device) for t in [val_seq, val_mask, val_y]]
        # Deactivate autograd
        with torch.no_grad():
            # Model predictions
            preds = model(sent_id, mask)
            # Compute the validation loss between actual and predicted values
            loss =  CEloss(preds, torch.max(labels, 1)[1]) 
            #loss = cross_entropy(preds,labels)
            total_loss = total_loss + loss.item()

    # Compute the validation loss of the epoch
    avg_loss = total_loss / len(val_dataloader) 

    return avg_loss

In [ ]:
batch = next(iter(train_dataloader))

train_seq, train_mask, train_y = tokenize_batch(batch)
# Push to device
sent_id, mask, labels = [t.to(device) for t in [train_seq, train_mask, train_y]]

# Get predictions
preds = model(sent_id, mask)

In [ ]:
# Epochs
epochs = 1

# Init loss
best_valid_loss = float('inf')

# data lists
train_losses=[]
valid_losses=[]

# Loop over epochs
for epoch in range(epochs):
     
    print('\n Epoch {:} / {:}'.format(epoch + 1, epochs))
    
    # Train model
    train_loss = train() 
    # Evaluate model
    #valid_loss  = evaluate()
        
    # Append training and validation loss
    train_losses.append(train_loss)
    #valid_losses.append(valid_loss)
    
    print(f'\nTraining Loss: {train_loss:.6f}')
    #print(f'Validation Loss: {valid_loss:.6f}')

In [87]:
strings = """
Large tree up to 18–25 m. high; crown rounded, with pendulous branchlets; bark brown to almost black, deeply longitudinally fissured.
"""
#

#string_list = ['spirally arranged leaves',
#              'bark grey-brown']

#strings = '. '.join(string_list)

In [88]:
tokens = tokenizer(strings, return_tensors="pt", truncation=True)

In [89]:
outputs = model(tokens['input_ids'], tokens['attention_mask'])

In [90]:
exps = torch.exp(outputs)
# Get class
span_class = exps.argmax(1).item()

In [91]:
exps = exps.detach().numpy()

In [92]:
'''
n=10

numbers = np.squeeze(exps)
idx = np.argpartition(numbers, -n)[-n:]
indices = idx[np.argsort((-numbers)[idx])]


#numbers[::-1].sort()
print(indices)
print(numbers[0:n])
'''

'\nn=10\n\nnumbers = np.squeeze(exps)\nidx = np.argpartition(numbers, -n)[-n:]\nindices = idx[np.argsort((-numbers)[idx])]\n\n\n#numbers[::-1].sort()\nprint(indices)\nprint(numbers[0:n])\n'

In [93]:
# Encoder 
lb = LabelBinarizer()
# Init dict
datadict = {}
# Load the pickle list
data_files = glob.glob(root + 'subset*.pkl')
for data_file in data_files:
    # Open the dict and update
    datadict.update(pickle.load(open(data_file, 'rb')))

# Get keys and encode them
keys = np.array([key for key in datadict.keys()])
#print(len(keys))
keys_encoded = lb.fit_transform(keys)

In [94]:
lb.inverse_transform(exps, 0.6)

array(['Bobgunnia madagascariensis'], dtype='<U37')

In [96]:
#datadict['Bobgunnia madagascariensis']